### 합성곱/풀링 구현하기


In [3]:
import numpy as np

x = np.random.rand(10, 1, 28, 28) # 무작위의 4차원 데이터 생성
print(x.shape) # (채널1개, 세로 28픽셀, 가로 28픽셀)의 데이터가 10개 있다는 의미

print(x[0].shape) # 1번째 데이터의 크기
print(x[1].shape) # 2번째 데이터의 크기

(10, 1, 28, 28)
(1, 28, 28)
(1, 28, 28)


### im2col(image to column)으로 데이터 전개하기

im2col은 입력 데이터를 필터링 하기 좋게 전개하는 함수입니다. 3차원 입력 데이터에 im2col을 적용하면 2차원 행렬로 바뀝니다. 아래는 그 예시입니다.

![screensh](../../screenshots/im2col.png)

im2col로 입력 데이터를 전개한 다음에는 합성곱 계층의 필터를 1열로 전개하고, 두 행렬의 곱을 계산합니다. <br>
출력한 결과는 2차원 행렬입니다. CNN은 데이터를 4차원 배열로 저장하므로 2차원인 출력 데이터를 4차원으로 변형합니다.

### 합성곱 계층 구현하기